In [1]:
from wfm.spherical_mesh import *
from timeit import timeit

In [ ]:
2**48 # possible vocab size

281474976710656

In [3]:
mesh = SphericalMesh()
type(mesh.points), mesh.points.shape, mesh.points.dtype

(numpy.ndarray, (100, 360, 180, 3), dtype('float32'))

In [4]:
torch_mesh = SphericalMesh(library = "torch")
type(torch_mesh.points), torch_mesh.points.shape, torch_mesh.points.dtype

(torch.Tensor, torch.Size([100, 360, 180, 3]), torch.float32)

In [5]:
jax_mesh = SphericalMesh(library = "jax")
type(jax_mesh.points), jax_mesh.points.shape, jax_mesh.points.dtype

(jaxlib.xla_extension.ArrayImpl, (100, 360, 180, 3), dtype('float32'))

In [ ]:
timeit("""SphericalMesh(library = 'torch')""", number = 100, globals = {"SphericalMesh": SphericalMesh})

11.557535199986887

---

In [1]:
from wfm.input_processor import *

c:\Users\LENOVO\AppData\Local\pypoetry\Cache\virtualenvs\wfm-ErW5Bk3W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'pd' is not defined

In [ ]:
tokenizer = BinaryTokenizer(base_tokenizer, base_embedder)